In [1]:
import pyDOE
import pandas as pd
import numpy as np
import math
import os
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from scipy.optimize import Bounds
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from collections import namedtuple
from matplotlib import cm
import cma
from scipy.optimize import minimize
from scipy.optimize import Bounds
import sys
import bbobbenchmarks as bn

## Benchmark Functions for Optimization

In [2]:
''' F2 '''
def F2(X):
    f = bn.F2()
    X = np.array(X)
    return f(X)

''' F3 '''
def F3(X):
    f = bn.F3()
    X = np.array(X)
    return f(X)

''' F7 '''
def F7(X):
    f = bn.F7()
    X = np.array(X)
    return f(X)

''' F9 '''
def F9(X):
    f = bn.F9()
    X = np.array(X)
    return f(X)

''' F10 '''
def F10(X):
    f = bn.F10()
    X = np.array(X)
    return f(X)

''' F13 '''
def F13(X):
    f = bn.F13()
    X = np.array(X)
    return f(X)

''' F15 '''
def F15(X):
    f = bn.F15()
    X = np.array(X)
    return f(X)

''' F16 '''
def F16(X):
    f = bn.F16()
    X = np.array(X)
    return f(X)

''' F20 '''
def F20(X):
    f = bn.F20()
    X = np.array(X)
    return f(X)

''' F24 '''
def F24(X):
    f = bn.F24()
    X = np.array(X)
    return f(X)

## Helper Functions

In [3]:
ValueRange = namedtuple('ValueRange', ['min', 'max'])

def determinerange(values):
    """Determine the range of values in each dimension"""
    return ValueRange(np.min(values, axis=0), np.max(values, axis=0))


def linearscaletransform(values, *, range_in=None, range_out=ValueRange(0, 1), scale_only=False):
    """Perform a scale transformation of `values`: [range_in] --> [range_out]"""

    if range_in is None:
        range_in = determinerange(values)
    elif not isinstance(range_in, ValueRange):
        range_in = ValueRange(*range_in)

    if not isinstance(range_out, ValueRange):
        range_out = ValueRange(*range_out)

    scale_out = range_out.max - range_out.min
    scale_in = range_in.max - range_in.min

    if scale_only:
        scaled_values = (values / scale_in) * scale_out
    else:
        scaled_values = (values - range_in.min) / scale_in
        scaled_values = (scaled_values * scale_out) + range_out.min

    return scaled_values

''' Latin HyperCube Sampling Design of Experiment '''
def DOE(n_obs, dim):
    np.random.seed(0)
    lhd = pyDOE.lhs(n=dim, samples=n_obs, criterion='m')
    X = [lhd[:,idx] for idx in range(dim)]
    return X

''' Generate Training Data using LHD along side the Output'''
def generate_training_data(n_obs, dim):
    X = DOE(n_obs, dim)
    X  = [ linearscaletransform(X[idx] , range_out=(-5,5)) for idx in range(dim) ]
    X = [ X[idx].reshape(n_obs,1) for idx in range(len(X)) ]
    f2_evaluation , f3_evaluation ,  f7_evaluation ,f9_evaluation , f10_evaluation , f13_evaluation , f15_evaluation , f16_evaluation , f20_evaluation , f24_evaluation= np.zeros(X[0].shape[0]) , np.zeros(X[0].shape[0]), np.zeros(X[0].shape[0]) , np.zeros(X[0].shape[0]), np.zeros(X[0].shape[0]), np.zeros(X[0].shape[0]), np.zeros(X[0].shape[0]), np.zeros(X[0].shape[0]), np.zeros(X[0].shape[0]), np.zeros(X[0].shape[0])
    for i in range(X[1].shape[0]):
        f2_evaluation [i], f3_evaluation [i], f7_evaluation [i],f9_evaluation [i], f10_evaluation [i], f13_evaluation [i], f15_evaluation [i], f16_evaluation [i], f20_evaluation [i] , f24_evaluation [i]= F2(np.concatenate(X, 1)[i]) , F3(np.concatenate(X, 1)[i]), F7(np.concatenate(X, 1)[i]), F9(np.concatenate(X, 1)[i]), F10(np.concatenate(X, 1)[i]), F13(np.concatenate(X, 1)[i]), F15(np.concatenate(X, 1)[i]), F16(np.concatenate(X, 1)[i]), F20(np.concatenate(X, 1)[i]), F24(np.concatenate(X, 1)[i])
    train_2 , train_3 , train_7, train_9, train_10, train_13 , train_15, train_16, train_20, train_24 = pd.DataFrame() , pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame()
    for i in range(len(X)):
        train_2 [str('X'+str(i+1))] , train_3 [str('X'+str(i+1))], train_7 [str('X'+str(i+1))], train_9 [str('X'+str(i+1))], train_10 [str('X'+str(i+1))], train_13 [str('X'+str(i+1))], train_15 [str('X'+str(i+1))], train_16 [str('X'+str(i+1))], train_20 [str('X'+str(i+1))], train_24 [str('X'+str(i+1))] = X[i].reshape(n_obs,) , X[i].reshape(n_obs,), X[i].reshape(n_obs,), X[i].reshape(n_obs,), X[i].reshape(n_obs,), X[i].reshape(n_obs,), X[i].reshape(n_obs,), X[i].reshape(n_obs,), X[i].reshape(n_obs,), X[i].reshape(n_obs,)
    train_2 ['Y'] , train_3 ['Y'], train_7 ['Y'], train_9 ['Y'], train_10 ['Y'], train_13 ['Y'], train_15 ['Y'], train_16 ['Y'], train_20 ['Y'], train_24 ['Y'] = pd.Series(f2_evaluation) , pd.Series(f3_evaluation), pd.Series(f7_evaluation), pd.Series(f9_evaluation), pd.Series(f10_evaluation), pd.Series(f13_evaluation), pd.Series(f15_evaluation), pd.Series(f16_evaluation), pd.Series(f20_evaluation), pd.Series(f24_evaluation)
    train_2.to_csv('Training_Data_Sets\\train_2_'+str(n_obs)+'Samples.csv') , train_3.to_csv('Training_Data_Sets\\train_3_'+str(n_obs)+'Samples.csv'), train_7.to_csv('Training_Data_Sets\\train_7_'+str(n_obs)+'Samples.csv'), train_9.to_csv('Training_Data_Sets\\train_9_'+str(n_obs)+'Samples.csv'), train_10.to_csv('Training_Data_Sets\\train_10_'+str(n_obs)+'Samples.csv'), train_13.to_csv('Training_Data_Sets\\train_13_'+str(n_obs)+'Samples.csv'), train_15.to_csv('Training_Data_Sets\\train_15_'+str(n_obs)+'Samples.csv'), train_16.to_csv('Training_Data_Sets\\train_16_'+str(n_obs)+'Samples.csv'), train_20.to_csv('Training_Data_Sets\\train_20'+str(n_obs)+'Samples.csv'), train_24.to_csv('Training_Data_Sets\\train_24'+str(n_obs)+'Samples.csv')
    return train_2 ,train_3, train_7, train_9, train_10, train_13, train_15, train_16, train_20, train_24

''' Generate Test Data using LHD along side the Output'''
def generate_test_data(n_obs,dim):
    X = DOE(n_obs, dim)
    X  = [ linearscaletransform(X[idx] , range_out=(-5,5)) for idx in range(dim) ]
    X = [ X[idx].reshape(n_obs,1) for idx in range(len(X)) ]
    f2_evaluation , f3_evaluation ,  f7_evaluation ,f9_evaluation , f10_evaluation , f13_evaluation , f15_evaluation , f16_evaluation , f20_evaluation , f24_evaluation= np.zeros(X[0].shape[0]) , np.zeros(X[0].shape[0]), np.zeros(X[0].shape[0]) , np.zeros(X[0].shape[0]), np.zeros(X[0].shape[0]), np.zeros(X[0].shape[0]), np.zeros(X[0].shape[0]), np.zeros(X[0].shape[0]), np.zeros(X[0].shape[0]), np.zeros(X[0].shape[0])
    for i in range(X[1].shape[0]):
        f2_evaluation [i] , f3_evaluation [i], f7_evaluation [i],f9_evaluation [i], f10_evaluation [i], f13_evaluation [i], f15_evaluation [i] , f16_evaluation [i], f20_evaluation [i], f24_evaluation [i]= F2(np.concatenate(X, 1)[i]) , F3(np.concatenate(X, 1)[i]), F7(np.concatenate(X, 1)[i]), F9(np.concatenate(X, 1)[i]), F10(np.concatenate(X, 1)[i]), F13(np.concatenate(X, 1)[i]), F15(np.concatenate(X, 1)[i]), F16(np.concatenate(X, 1)[i]), F20(np.concatenate(X, 1)[i]), F24(np.concatenate(X, 1)[i])
    test_2 , test_3 , test_7, test_9, test_10, test_13 , test_15, test_16, test_20, test_24 = pd.DataFrame() , pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame()
    for i in range(len(X)):
        test_2 [str('X'+str(i+1))] , test_3 [str('X'+str(i+1))], test_7 [str('X'+str(i+1))], test_9 [str('X'+str(i+1))], test_10 [str('X'+str(i+1))], test_13 [str('X'+str(i+1))], test_15 [str('X'+str(i+1))], test_16 [str('X'+str(i+1))], test_20 [str('X'+str(i+1))], test_24 [str('X'+str(i+1))] = X[i].reshape(n_obs,) , X[i].reshape(n_obs,), X[i].reshape(n_obs,), X[i].reshape(n_obs,), X[i].reshape(n_obs,), X[i].reshape(n_obs,), X[i].reshape(n_obs,), X[i].reshape(n_obs,), X[i].reshape(n_obs,), X[i].reshape(n_obs,)
    test_2 ['Y'] , test_3 ['Y'], test_7 ['Y'], test_9 ['Y'], test_10 ['Y'], test_13 ['Y'], test_15 ['Y'], test_16 ['Y'], test_20 ['Y'], test_24 ['Y'] = pd.Series(f2_evaluation) , pd.Series(f3_evaluation), pd.Series(f7_evaluation), pd.Series(f9_evaluation), pd.Series(f10_evaluation), pd.Series(f13_evaluation), pd.Series(f15_evaluation), pd.Series(f16_evaluation), pd.Series(f20_evaluation), pd.Series(f24_evaluation)
    test_2.to_csv('Test_Data_Sets\\test_2_'+str(n_obs)+'Samples.csv') , test_3.to_csv('Test_Data_Sets\\test_3_'+str(n_obs)+'Samples.csv'), test_7.to_csv('Test_Data_Sets\\test_7_'+str(n_obs)+'Samples.csv'), test_9.to_csv('Test_Data_Sets\\test_9_'+str(n_obs)+'Samples.csv'), test_10.to_csv('Test_Data_Sets\\test_10_'+str(n_obs)+'Samples.csv'), test_13.to_csv('Test_Data_Sets\\test_13_'+str(n_obs)+'Samples.csv'), test_15.to_csv('Test_Data_Sets\\test_15_'+str(n_obs)+'Samples.csv'), test_16.to_csv('Test_Data_Sets\\test_16_'+str(n_obs)+'Samples.csv'), test_20.to_csv('Test_Data_Sets\\test_20_'+str(n_obs)+'Samples.csv'), test_24.to_csv('Test_Data_Sets\\test_24_'+str(n_obs)+'Samples.csv')
    return test_2 ,test_3, test_7, test_9, test_10, test_13, test_15, test_16, test_20, test_24

In [4]:
dim = 100
multiple = [20]
for i in range(len(multiple)):
    train_2,train_3,train_7,train_9,train_10,train_13,train_15,train_16,train_20,train_24 = generate_training_data(dim * multiple[i], dim)
test_2,test_3,test_7,test_9,test_10,test_13,test_15,test_16,test_20,test_24 = generate_test_data(int(dim * 20 * 0.2), dim)
data_2 , data_3, data_7, data_9, data_10, data_13, data_15, data_16, data_20, data_24 = pd.concat([train_2, test_2]) , pd.concat([train_3, test_3]), pd.concat([train_7, test_7]), pd.concat([train_9, test_9]), pd.concat([train_10, test_10]), pd.concat([train_13, test_13]), pd.concat([train_15, test_15]), pd.concat([train_16, test_16]), pd.concat([train_20, test_20]), pd.concat([train_24, test_24])
print ('No. of Duplicate rows between Train & Test Data Set are: '+str(data_2.duplicated().sum()))
print ('No. of Duplicate rows between Train & Test Data Set are: '+str(data_3.duplicated().sum()))
print ('No. of Duplicate rows between Train & Test Data Set are: '+str(data_7.duplicated().sum()))
print ('No. of Duplicate rows between Train & Test Data Set are: '+str(data_9.duplicated().sum()))
print ('No. of Duplicate rows between Train & Test Data Set are: '+str(data_10.duplicated().sum()))
print ('No. of Duplicate rows between Train & Test Data Set are: '+str(data_13.duplicated().sum()))
print ('No. of Duplicate rows between Train & Test Data Set are: '+str(data_15.duplicated().sum()))
print ('No. of Duplicate rows between Train & Test Data Set are: '+str(data_16.duplicated().sum()))
print ('No. of Duplicate rows between Train & Test Data Set are: '+str(data_20.duplicated().sum()))
print ('No. of Duplicate rows between Train & Test Data Set are: '+str(data_24.duplicated().sum()))

No. of Duplicate rows between Train & Test Data Set are: 0
No. of Duplicate rows between Train & Test Data Set are: 0
No. of Duplicate rows between Train & Test Data Set are: 0
No. of Duplicate rows between Train & Test Data Set are: 0
No. of Duplicate rows between Train & Test Data Set are: 0
No. of Duplicate rows between Train & Test Data Set are: 0
No. of Duplicate rows between Train & Test Data Set are: 0
No. of Duplicate rows between Train & Test Data Set are: 0
No. of Duplicate rows between Train & Test Data Set are: 0
No. of Duplicate rows between Train & Test Data Set are: 0
